In [ ]:
import os,time
import scipy.io
import scipy.stats
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print(tf.__version__)

In [ ]:
PHM_path = 'F:\\Data\\phm-ieee-2012-data-challenge-dataset-master\\mat'
PHM_bearing_files = [os.path.join(PHM_path,file) for file in os.listdir(PHM_path)]

def get_FPT(h2):
    '''h2：h.rehsape(-1,2560)的结果'''
    kurt_list = []
    rms_list = []
    for i,row in enumerate(h2):
        kurt = scipy.stats.kurtosis(row)
        kurt_list.append(kurt)
        rms = np.mean(row**2)**0.5
        rms_list.append(rms)
        weight = np.concatenate([np.linspace(5,   4.5, 100),
                                 np.linspace(4.5, 4,   500),
                                 np.linspace(4,   3,   2000),
                                 np.linspace(3,   3,   3000)])
        w = weight[i]
        kurt_c = kurt > np.mean(kurt_list)+w*np.std(kurt_list)
        rms_c  = rms  > np.mean(rms_list) +w*np.std(rms_list)
        if kurt_c and rms_c:
            break
    return i

def mat_to_arr(file):
    h = scipy.io.loadmat(file)['h'].reshape(-1)
    h2 = h.reshape(-1,2560)
    kurtosis = np.array( [scipy.stats.kurtosis(i) for i in h2] )
    rms = np.array( [np.mean(i**2)**0.5 for i in h2] )
    rms = np.convolve(rms,[0.3,0.4,0.3],mode='same')
    ma = np.array( [np.max(np.abs(i)) for i in h2] )
    FPT = get_FPT(h2)
    return h,FPT,kurtosis,rms,ma

In [ ]:
for file in PHM_bearing_files[:5]:

    h,FPT,kurtosis,rms,ma = mat_to_arr(file)
    tlong = np.linspace(0,len(rms)*10,len(h))
    tshort = np.linspace(0,len(rms)*10,len(rms))
    FPT = FPT*10
            
    
    plt.figure(figsize=(5,6))
    plt.title(file[-14:-4])
    
    plt.subplot(311)
    plt.plot(tlong,h,'b',label='Signal')
    plt.plot([FPT,FPT],[min(h),max(h)],'r-.',label='FPT')
    # 设置字体
    plt.yticks(fontproperties = 'Times New Roman', size = 10)
    plt.xticks(fontproperties = 'Times New Roman', size = 10)
#     plt.xlabel('Time (s)', fontproperties = 'Times New Roman', fontsize=12)
    plt.ylabel('Signal (g)', fontproperties = 'Times New Roman', fontsize=12)
#     plt.legend(loc='upper left')
    
    plt.subplot(312)
    plt.plot(tshort,kurtosis,'b',label='Kurtosis')
    plt.plot([FPT,FPT],[min(kurtosis),max(kurtosis)],'r-.',label='FPT')
    # 设置字体
    plt.yticks(fontproperties = 'Times New Roman', size = 10)
    plt.xticks(fontproperties = 'Times New Roman', size = 10)
#     plt.xlabel('Time (s)', fontproperties = 'Times New Roman', fontsize=12)
    plt.ylabel('Kurtosis', fontproperties = 'Times New Roman', fontsize=12)
#     plt.legend(loc='upper left')
    
    plt.subplot(313)
    plt.plot(tshort,rms,'b',label='RMS')
    plt.plot([FPT,FPT],[min(rms),max(rms)],'r-.',label='FPT')
    # 设置字体
    plt.yticks(fontproperties = 'Times New Roman', size = 10)
    plt.xticks(fontproperties = 'Times New Roman', size = 10)
    plt.xlabel('Time (s)', fontproperties = 'Times New Roman', fontsize=12)
    plt.ylabel('RMS (g)', fontproperties = 'Times New Roman', fontsize=12)
#     plt.legend(loc='upper left')
    
    
#     plt.subplot(414)
#     plt.plot(tshort,ma,'b',label='MAX')
#     plt.plot([FPT,FPT],[min(ma),max(ma)],'r-.',label='FPT')
#     plt.xlabel('Time (s)', fontproperties = 'Times New Roman', fontsize=25)
#     plt.ylabel('MAX (g)', fontproperties = 'Times New Roman', fontsize=25)
#     plt.legend(loc='upper left')
    
#     plt.savefig('./FPT_Selection/'+file[-14:-4]+'_FPT_Selection',dpi=200)
    plt.show()

In [ ]:
# h,FPT,kurtosis,rms,ma = mat_to_arr(PHM_bearing_files[3]) # 1-4
# tlong = np.linspace(0,len(rms)*10,len(h))
# tshort = np.linspace(0,len(rms)*10,len(rms))
# FPT = FPT*10

# plt.figure(figsize=(10,8))
# plt.subplot(211)
# plt.plot(tlong,h,'b',label='Vibration signal')
# plt.plot([FPT,FPT],[-60,60],'r--',lw=2,label='FPT')
# plt.legend(loc='upper left')
# plt.ylim(-50,50)
# # plt.xlabel('Time (s)',fontproperties = 'Times New Roman', fontsize=15)
# plt.ylabel('Amplitude (g)',fontproperties = 'Times New Roman', fontsize=15)
# plt.yticks(fontproperties = 'Times New Roman', size = 14)
# plt.xticks(fontproperties = 'Times New Roman', size = 14)

# plt.subplot(212)
# plt.plot(tshort,rms,'b',label='RMS')
# plt.plot([FPT,FPT],[-5,15],'r--',lw=2,label='FPT')
# plt.plot([3]*len(ma)*10,'g--',label='Failure threshold')
# plt.legend(loc='upper left')
# plt.ylim(-2,12)
# plt.xlabel('Time (s)',fontproperties = 'Times New Roman', fontsize=15)
# plt.ylabel('RMS (g)',fontproperties = 'Times New Roman', fontsize=15)
# plt.yticks(fontproperties = 'Times New Roman', size = 14)
# plt.xticks(fontproperties = 'Times New Roman', size = 14)
# plt.savefig(fname='two_stage_rms', dpi=300,)
# plt.show()


# DataGenerator

In [ ]:
# test_bearing_IPT = [18010,  11380,  23010,  23010,  15010,
#                     12010,  6110,   20010,  5710,   1710,   3510]
# test_bearing_RUL = [5730,   339,    1610,   1460,   7570,
#                     7530,   1390,   3090,   1290,   580,    820]
# test_bearing = ['1_'+str(i) for i in range(3,7+1)] + ['2_'+str(i) for i in range(3,7+1)] + ['3_3']

test_bearing = ['1_1','1_3','1_4']

test_bearing_files = []
for f in PHM_bearing_files:
    if f[-7:-4] in test_bearing:
        test_bearing_files.append(f)
test_bearing_files

In [ ]:
def get_fea_FPT(bearing_files):
    feature_list = []
    FPT_list = []
    for i,file in enumerate(bearing_files):
        h,FPT,kurtosis,rms,ma = mat_to_arr(file)
        h2 = h.reshape(-1,2560)        
        fea = np.concatenate(  [   rms[:].reshape(-1,1),
                                       ma[:].reshape(-1,1),         ],  axis=1)
        print(file[-14:-4], h2.shape, fea.shape,FPT, sep='\t')
        feature_list.append(fea)
        FPT_list.append(FPT)

    return feature_list,FPT_list

'''
Bearing1_1	(2803, 2560)	(2803, 2)	2117	2766
Bearing1_3	(2375, 2560)	(2375, 2)	1681	2325
Bearing1_4	(1428, 2560)	(1428, 2)	1085	1140'''
feature_list,FPT_list = get_fea_FPT(test_bearing_files)
len(feature_list), feature_list[0].shape

In [ ]:
# '''这里使用乃鹏论文中的FPT，不用复现乃鹏的方法，只需要让梯度优化的指数模型使用和乃鹏一样的数据即可进行对比'''
FPT_list = [1463,1644,1091]
FT_list = [2763,2287,1139]

# 构建模型

In [ ]:
zz = np.linspace(-100,2000,100)

# a1 = np.exp(zz)
# a2 = -1*np.exp(zz)
# a3 = np.exp(-1*zz)
# a4 = -1*np.exp(-1*zz)


plt.figure(figsize=(12,6))
plt.plot(zz, np.exp(0.001*zz), 'r-', label='1')
plt.plot(zz, 0.001*np.exp(0.01*zz), 'r--', label='2')

plt.plot(zz, np.exp(0.002*zz), 'b-.', label='3')
plt.plot(zz, np.exp(0.004*zz), 'b-*', label='4')
plt.legend()
plt.ylim(-5,50)
plt.grid()
plt.xlabel('input', fontproperties = 'Times New Roman', size = 15)
plt.ylabel('output', fontproperties = 'Times New Roman', size = 15)

plt.xticks(fontproperties = 'Times New Roman', size = 14)
plt.yticks(fontproperties = 'Times New Roman', size = 14)
# plt.savefig('ab.png',dpi=300)
plt.show()



In [ ]:
class EModel(tf.Module):
    def __init__(self,init=[1.0,0.005,0.0,0.0],**kwargs):
        super().__init__(**kwargs)
#         inita = 1.0
#         initb = 0.005
#         initc = 0.0
#         initd = 0.0
        inita = tf.constant(init[0],dtype=tf.float32)
        initb = tf.constant(init[1],dtype=tf.float32)
        print('initb',initb)
        initc = tf.constant(init[2],dtype=tf.float32)
        initd = tf.constant(init[3],dtype=tf.float32)
#         self.a = tf.Variable(inita, name='a')
#         self.b = tf.Variable(initb,name='b')
#         self.c = tf.Variable(initc, name='c')
#         self.d = tf.Variable(initd, name='d')
        
        self.a = tf.Variable(tf.math.log(inita), name='a',dtype=tf.float32)
        self.b = tf.Variable(tf.math.log(initb), name='b',dtype=tf.float32)
        self.c = tf.Variable(initc, name='c',dtype=tf.float32)
        self.d = tf.Variable(initd, name='d',dtype=tf.float32)
        print('Init:',self.__call__(100.0).numpy(),self.a.numpy(),tf.exp(self.a).numpy())
        
        
        
    def __call__(self, x):
        x = tf.cast(x,dtype=tf.float32)
        a,b,c,d = self.a, self.b, self.c, self.d
        a = tf.exp(a)
        b = tf.exp(b)+0.0005
#         c = tf.exp(c)
#         d = tf.exp(d)
        y = a*tf.exp(b*x+d) +c

        return y
init_model = EModel()
zz = np.linspace(-200,1000,1000)
plt.figure(figsize=(10,3))
plt.plot(zz, init_model(zz))
plt.xlim(-200,1000)
plt.ylim(-1,10)
plt.grid()

In [ ]:
# zz = np.linspace(-5,5,100)

# a1 = np.exp(zz)
# a2 = -1*np.exp(zz)
# a3 = np.exp(-1*zz)
# a4 = -1*np.exp(-1*zz)


# plt.figure(figsize=(6,6))
# plt.plot(zz, a1, 'r-', label='a>0, b>0')
# plt.plot(zz, a2, 'r--', label='a<0, b>0')

# plt.plot(zz, a3, 'b-.', label='a>0, b<0')
# plt.plot(zz, a4, 'b-*', label='a<0, b<0')
# plt.legend()
# plt.ylim(-5,5)
# plt.grid()
# plt.xlabel('input', fontproperties = 'Times New Roman', size = 15)
# plt.ylabel('output', fontproperties = 'Times New Roman', size = 15)

# plt.xticks(fontproperties = 'Times New Roman', size = 14)
# plt.yticks(fontproperties = 'Times New Roman', size = 14)
# # plt.savefig('ab.png',dpi=300)
# plt.show()

# weight_y = np.linspace(-0.5,2,100)
# weight_x = np.arange(1,len(weight_y)+1)

# plt.plot(weight_x, weight_y)
# # plt.ylim(0,10)

In [ ]:
def plot_predict_curve(init_model,trained_model,history_time,history_data,all_time,all_data,ckpt,
                       thd,predict_FT=None,fn=None,xlim=None,):
    plt.figure(figsize=(10,6))
    # 检查点 前后
    plt.scatter(all_time*10, all_data, c="c", label='After checkpoint')
    plt.scatter(history_time*10, history_data, c="b", label='Before checkpoint')
    
    
    # FPT,FT,Failure threshold
    plt.plot([FPT*10,FPT*10],[0,10],'g--',label='FPT')
    plt.plot([FT*10,FT*10],[0,10],'k--',label='FT')
    if predict_FT is not None:
        plt.plot([thd]*5000*10,'k',label='Failure threshold')
        plt.plot([predict_FT*10,predict_FT*10],[0,10],'y',label='Predicted FT')
    else:
        plt.plot([thd]*5000*10,'k',label='Failure threshold')

    
    # 预测曲线
    t = np.arange(-1000,2000) # 用于输入模型进行预测
    t2 = (t + FPT)*10  # 整体后移动到开始预测的地方，用于画图，FPT  = 模型的陵石刻
    plt.plot(t2, init_model(t),    "m--", linewidth=3, label='Prediction before training')
    if trained_model is not None:
        plt.plot(t2, trained_model(t), "r--", linewidth=3, label='Prediction after training')
        
    
    
    # 设置字体
    plt.xlabel('Time (s)',fontproperties = 'Times New Roman', fontsize=20)
    plt.ylabel('RMS (g)',fontproperties = 'Times New Roman', fontsize=20)
    plt.yticks(fontproperties = 'Times New Roman', size = 20)
    plt.xticks(fontproperties = 'Times New Roman', size = 20)
    
    # 坐标轴范围
    if xlim is not None:
        plt.xlim(xlim)
    plt.ylim(0,5)
    
    plt.legend(loc='upper left')
    if fn is not None:
        plt.savefig(fname=fn, dpi=300,)
    plt.show()
    
    
def get_init_model(data):
    st = np.min(data)
    k = 100 if len(data)>100 else 5
    en = np.mean(data[-k:])
    a = 0.5
    c = st-a # 不管啥时候，a+c=y[0]
    b = np.log((en-c)/a)/len(data)
    print(1111111,b)
    d = 0.0
    init = [a,b,c,d]
    model = EModel(init) 
    return model

threshold = [1.6,1.5,3.0]


In [ ]:
for n in range(3):
    # 获取数据
    data  = feature_list[n][:,0]
#     cov = [1/3 for i in range(10)]
#     data = np.convolve(rms,cov,mode='same')
    
    FPT = FPT_list[n]
    FT = FT_list[n]
    thd = threshold[n]
    if n==0: FPT=2117

    # 划定检查点
    max_RUL = FT - FPT
    checkpoint = FPT + int(max_RUL*0.99)
#     checkpoint = FPT+1

    # 检查点前的数据
    history_data = data[:checkpoint+1]
    history_time = np.arange(len(history_data))

    # 所有数据
    all_data = data
    all_time = np.arange(len(all_data))

    # 初始化模型
    init_model = get_init_model(history_data[FPT:])


    plot_predict_curve(init_model,None,history_time,history_data,all_time,all_data,ckpt=checkpoint,
                       thd=thd,fn=None,xlim=[10*(FPT-60),10*(FT+50)])    
    plot_predict_curve(init_model,None,history_time,history_data,all_time,all_data,ckpt=checkpoint,
                       thd=thd,fn=None,xlim=None) 

In [ ]:
def loss(y_true, y_pred):
    w = np.linspace(0.5,2.0,len(y_true))
#     w = np.exp(w)
    weight = w * y_true
#     mse = tf.keras.losses.mse(y_true,y_pred)
    se = tf.square(y_true - y_pred) 
    se = se * weight
    mse = tf.reduce_mean(se)
    return mse


def train_step(model, x_true,y_true):
    lr = 5e-5
    opt = tf.keras.optimizers.Adam(lr)
    with tf.GradientTape() as gt:
        y_pred  = model(x_true)
        current_train_step_loss = loss(y_true, y_pred)
    gradients = gt.gradient(current_train_step_loss, model.trainable_variables) # 梯度
    opt.apply_gradients(zip(gradients, model.trainable_variables)) # 进行优化
    return current_train_step_loss

def train_model(history_data):
    # history_data 是检查点之前的所有rms，而不是FPT之前的，FPT < 检查点 < FT
    # 训练时，使用FPT前50到检查点之间的数据，且必须用这个数据先生成时间
    y_true = history_data[FPT:]
    x_true = np.arange(len(y_true))
    print("train_model 入口  history_data.shape=%d,  y_true.shape=%d,  x_true[0]=%d, x_true[-1]=%d" %
                  (len(history_data),len(y_true),x_true[0],x_true[-1]))

    model  = get_init_model(y_true)
    aa,bb,cc,dd,losses =[], [], [], [], []
    for step in range(10000):
        train_step(model, x_true,y_true )
        aa.append(model.a.numpy())
        bb.append(model.b.numpy())
        cc.append(model.c.numpy())
        dd.append(model.d.numpy())
        y_pred = model(x_true)
        losses.append(loss(y_true, y_pred).numpy())
        
        if step%100==0:
            print("step  %3d: a=%1.8f    b=%1.8f    c=%1.8f    d=%1.8f,   loss=%2.8f" %
                  (step, aa[-1], bb[-1], cc[-1], dd[-1], losses[-1]))
            print("step  %3d: a=%1.8f    b=%1.8f    c=%1.8f    d=%1.8f,   loss=%2.8f \n" %
                  (step, np.exp(aa[-1]), np.exp(bb[-1]), np.exp(cc[-1]), np.exp(dd[-1]), losses[-1]))
        min_step = 50 
        max_step = 200
        if step>max_step or step>=min_step and max(losses[-20:-1] )<=0.2:
            break
        log = np.stack([aa,bb,cc,dd,losses] )
    return get_init_model(y_true),model, log

init_model,trained_model,train_log = train_model(history_data)

In [ ]:
plot_predict_curve(init_model,trained_model,history_time,history_data,all_time,all_data,ckpt=checkpoint,
                   thd=thd,fn='train_process',xlim=[(FPT-int((FT-FPT)*1.5))*10,(FT+int((FT-FPT)*0.5))*10])

def show_and_save(data,index=None,xlabel='x-axis',ylabel='y-axis',filename=None,ylim=None):
    plt.figure(figsize=(4,2))
    if index is None: index = np.arange(len(data))
    plt.plot(index,data)
    plt.yticks(np.array(np.linspace(min(data)*10000, max(data)*10000, 2),dtype=np.int)/10000)
    plt.yticks(fontproperties = 'Times New Roman', size = 20)
    plt.xticks(fontproperties = 'Times New Roman', size = 20)
#     plt.xlabel(xlabel, fontproperties = 'Times New Roman', fontsize=20)
#     plt.ylabel(ylabel, fontproperties = 'Times New Roman', fontsize=20)
    if ylim is not None: plt.ylim(ylim)
    plt.savefig(fname=filename + '.png', dpi=150,)
    plt.show()

show_and_save(data=train_log[0],xlabel='Step',ylabel='a',filename='a',ylim=None)
show_and_save(data=train_log[1],xlabel='Step',ylabel='b',filename='b',ylim=None)
show_and_save(data=train_log[2],xlabel='Step',ylabel='c',filename='c')
show_and_save(data=train_log[3],xlabel='Step',ylabel='d',filename='d')
show_and_save(data=train_log[4],xlabel='Step',ylabel='Loss',filename='loss')

In [ ]:
def get_predict_FT(history_data,trained_model,ckpt,threshold):
    # 1.因该从已知的历史数据的最后开始检查是否是FT
    # 2.构建的模型是从FPT开始的指数模型，FPT是模型的零时刻，
    print('2222')
    t = ckpt-FPT
    predictd_rms = trained_model(t).numpy()
    while predictd_rms<threshold:
        t = t+1
        predictd_rms = trained_model(t).numpy()
    t = t
    FT = t + FPT
    if FT<1 : FT=1
    print('233')
    return FT
        
predict_FT = get_predict_FT(history_data, trained_model,checkpoint,threshold=thd)
print(predict_FT)

plot_predict_curve(init_model,trained_model,history_time,history_data,all_time,all_data,ckpt=checkpoint,thd=thd,
                   predict_FT=predict_FT,fn='bearing1_3_RUL',
                   xlim=[(FPT-int((FT-FPT)*0.2))*10,(max(FT,predict_FT)+int((FT-FPT)*0.2))*10])

In [ ]:
def add_zero(num):
    s = str(num)
    for i in range(4-len(s)):
        s = '0'+s
    return s

add_zero(123)

In [ ]:
all_predictd_FT_list = []   # 所有轴承的预测
for n in range(len(feature_list)):
    # 获取数据
    data  = feature_list[n][:,0]
    FPT = FPT_list[n]
    FT = FT_list[n]
    thd=threshold[n]
    print('检查点-FPT',data[FT]-data[FPT])
    
    # 划定检查点
    d = (FT - FPT)//10
#     d = FT - FPT
    predictd_FT_list = []  # 一个轴承的预测
    for m in range(1,11):
        checkpoint = FPT + m*d
        if checkpoint==FT: break
        print('n,  m,  checkpoint,  FPT,  FT,  d')
        print(n,m,checkpoint,FPT,FT,d,sep='  ')
        # 检查点前的数据
        history_data = data[:checkpoint+1]  # 索引加一才可以取到checkpoint点
        history_time = np.arange(len(history_data))
        # 所有数据 画图用
        all_data = data
        all_time = np.arange(len(all_data))
        print(m,n,'history_data.shape',history_data.shape)
        # 训练模型  预测FT
        init_model,trained_model,train_log = train_model(history_data)
        predict_FT = get_predict_FT(history_data, trained_model, ckpt = checkpoint,threshold=thd)
        print('2222')
        t_RUL = FT-checkpoint
        p_RUL = predict_FT-checkpoint
        
        predictd_FT_list.append([checkpoint,t_RUL,p_RUL,])
        if checkpoint%20==0:
            plot_predict_curve(init_model,trained_model,history_time,history_data,all_time,all_data,
                               ckpt=checkpoint,thd=thd,predict_FT=predict_FT,
                               fn='./train/bearing_'+add_zero(n)+'_'+add_zero(m)
                                  +'_'+add_zero(checkpoint)+'_'+add_zero(t_RUL)+'_'+add_zero(p_RUL),
                               xlim=[(FPT-int((FT-FPT)*0.2))*10,(max(FT,predict_FT)+int((FT-FPT)*0.2))*10])
#         last_para = new_para
        
    all_predictd_FT_list.append(predictd_FT_list)
11

In [ ]:
all_predictd_FT_list

In [ ]:
x = np.array(all_predictd_FT_list[2][:])
print(x.shape)
for i in range(len(x)):
    t = x[i][1]
    p = x[i][2]
    print(i,t,p,1-abs(t-p)/t)
    print()

In [ ]:
result = np.array(all_predictd_FT_list)


In [ ]:
# np.save('result',result)

In [ ]:
np.arange(10,0,-1)

In [ ]:
def CRA(true,pred):
    K = len(true)
    w = np.arange(1,K+1)
    w = w / sum(w)
    RA = 1 - np.abs(true-pred)/(true+1e-6)
    CRA = np.sum(w * RA)
    plt.figure()
    plt.plot(RA)
    plt.ylim(-0.5,1.5)
    
    return CRA,np.mean(RA)#,RA[::100]

for i in range(3):
    result_i = np.array(all_predictd_FT_list[i])
    t = result_i[:,1]
    p = result_i[:,2]
    print(3333,CRA(t,p))

In [ ]:
s = np.array(all_predictd_FT_list[2])
s.shape,s[2].shape

In [ ]:

plt.figure(figsize=(10,5))

for i in range(3):
    x = np.array(all_predictd_FT_list[i])
    FT = FT_list[i]
    # ck + pred + true
    ckpt = x[:,0]
    predrul = x[:,1]
    truerul = x[:,2]
    
    predrms = ckpt + predrul
    truerms = ckpt + truerul
    
    plt.subplot(2,3,i+1)
    plt.plot(ckpt,truerms,label='RMS')
    plt.plot(ckpt,predrms,label='Predictd RMS')
    plt.legend()
    
    plt.subplot(2,3,i+3+1)
    plt.plot(ckpt,truerul,label='RUL')
    plt.plot(ckpt,predrul,label='Predictd RUL')
    plt.legend()
    
plt.show()

In [ ]:
for i in range(3):
    x = np.array(all_predictd_FT_list[i])
    FT = FT_list[i]
    # ck + pred + true
    ckpt = x[:,0]
    predrul = x[:,1]
    truerul = x[:,2]
    
    predrms = ckpt + predrul
    truerms = ckpt + truerul

    plt.figure(figsize=(8,4))
    plt.plot(ckpt*10,truerul*10,'b',label='RUL')
    plt.plot(ckpt*10,predrul*10,'r--',label='Predictd RUL')
    plt.legend()

    # plt.yticks(np.array(np.linspace(min(data), max(data), 3)*10000,dtype=np.int)/10000.)
    plt.yticks(fontproperties = 'Times New Roman', size = 15)
    plt.xticks(fontproperties = 'Times New Roman', size = 15)
    plt.xlabel('Time (s)', fontproperties = 'Times New Roman', fontsize=15)
    plt.ylabel('RUL (s)', fontproperties = 'Times New Roman', fontsize=15)
#     plt.ylim(0,800)
    plt.savefig(fname='RUL927prul_' + str(i), dpi=200,)
    plt.show()

In [ ]:
# result = {'result':np.array(all_predictd_FT_list),
#             'FPT':np.array(FPT_list),
#             'FT':np.array(FT_list)}
# result

In [ ]:
# scipy.io.savemat('result',result)